In [2]:
import os
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : C:\Users\karol\SCI_LEARN\Github\Klasyfikator
Directory name is : Klasyfikator


In [4]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('..\Share\db\db.sqlite')

df_from_db = pd.read_sql_query("SELECT text_review, rating_star FROM reviews", cnx)

In [5]:
df_from_db

,text_review,rating_star


In [ ]:
df_from_db['rating_star'] = df_from_db['rating_star'].astype(int)

In [ ]:
import numpy as np
np.random.seed(0)

df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.to_csv('./input_data-pl.csv', index=False, encoding='utf-8')

In [ ]:
import pandas as pd
df = pd.read_csv('./input_data-pl.csv', encoding='utf-8')

In [ ]:
df

In [ ]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)


In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stop = ['a','aby','ach','acz','aczkolwiek','aj','albo','ale','ależ','ani','aż','bardziej','bardzo','bez','bo','bowiem','by','byli','bym','bynajmniej','być','był','była','było','były','będzie','będą','cali','cała','cały','chce','choć','ci','ciebie','cię','co','cokolwiek','coraz','coś','czasami','czasem','czemu','czy','czyli','często','daleko','dla','dlaczego','dlatego','do','dobrze','dokąd','dość','dr','dużo','dwa','dwaj','dwie','dwoje','dzisiaj','dziś','gdy','gdyby','gdyż','gdzie','gdziekolwiek','gdzieś','go','godz','hab','i','ich','ii','iii','ile','im','inna','inne','inny','innych','inż','iv','ix','iż','ja','jak','jakaś','jakby','jaki','jakichś','jakie','jakiś','jakiż','jakkolwiek','jako','jakoś','je','jeden','jedna','jednak','jednakże','jedno','jednym','jedynie','jego','jej','jemu','jest','jestem','jeszcze','jeśli','jeżeli','już','ją','każdy','kiedy','kierunku','kilka','kilku','kimś','kto','ktokolwiek','ktoś','która','które','którego','której','który','których','którym','którzy','ku','lat','lecz','lub','ma','mają','mam','mamy','mało','mgr','mi','miał','mimo','między','mnie','mną','mogą','moi','moim','moja','moje','może','możliwe','można','mu','musi','my','mój','na','nad','nam','nami','nas','nasi','nasz','nasza','nasze','naszego','naszych','natomiast','natychmiast','nawet','nic','nich','nie','niech','niego','niej','niemu','nigdy','nim','nimi','nią','niż','no','nowe','np','nr','o','o.o.','obok','od','ok','około','on','ona','one','oni','ono','oraz','oto','owszem','pan','pana','pani','pl','po','pod','podczas','pomimo','ponad','ponieważ','powinien','powinna','powinni','powinno','poza','prawie','prof','przecież','przed','przede','przedtem','przez','przy','raz','razie','roku','również','sam','sama','się','skąd','sobie','sobą','sposób','swoje','są','ta','tak','taka','taki','takich','takie','także','tam','te','tego','tej','tel','temu','ten','teraz','też','to','tobie','tobą','toteż','totobą','trzeba','tu','tutaj','twoi','twoim','twoja','twoje','twym','twój','ty','tych','tylko','tym','tys','tzw','tę','u','ul','vi','vii','viii','vol','w','wam','wami','was','wasi','wasz','wasza','wasze','we','według','wie','wiele','wielu','więc','więcej','wszyscy','wszystkich','wszystkie','wszystkim','wszystko','wtedy','www','wy','właśnie','wśród','xi','xii','xiii','xiv','xv','z','za','zapewne','zawsze','zaś','ze','zeznowu','znowu','znów','został','zł','żaden','żadna','żadne','żadnych','że','żeby']


In [ ]:
def tokenizer(text):
    
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # pomija nagłówek
        j = 0
        for line in csv:
            try:
                j += 1
                text, label = line[:-3], int(line[-2])
                #print(text[:60], "label", label)
            except Exception as e:
                #j += 1
                #print("Linia",j,e, "Text:" ,text, "label:", label)

                pass
       
            yield text, label

In [ ]:
next(stream_docs(path='./input_data-pl.csv'))

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        #print("Blad: ", text[:60], "label", label)
        return None, None
    return docs, y

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='./input_data-pl.csv')

In [ ]:
import pyprind
pbar = pyprind.ProgBar(300)

classes = np.array([0, 1])
for _ in range(300):
    try:
        X_train, y_train = get_minibatch(doc_stream, size=1000)

        if not X_train:
            break

        X_train = vect.transform(X_train)

        clf.partial_fit(X_train, y_train, classes=classes)

        pbar.update()
        
    except Exception as e:
        
        #print("Blad:", e)
        pass
    
    
#print("======= X_test ========")
#i_train=1
#for tst in X_test:
#    print(i_train,")",tst)
#    i_train += 1

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=500)

#print("======= X_test ========")
#i_test=1
#for tst in X_test:
#    print(i_test,")",tst)
#    i_test += 1
    
#print("======= y_test ========")
#i_test=1
#for tst in y_test:
#    print(i_test,")",tst)
#    i_test += 1

X_test = vect.transform(X_test)




print('Dokładność: %.3f' % clf.score(X_test, y_test))

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
import pickle
import os

dest = os.path.join('pkl_objects')

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [ ]:
%%writefile ./vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))


def tokenizer(text):
   
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

In [6]:
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : C:\Users\karol\SCI_LEARN\Github\Klasyfikator
Directory name is : Klasyfikator


In [ ]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))


In [ ]:
import numpy as np
label = {0:'Negatywna', 1:'Pozytywna'}

example = ['Sklep jest super']
X = vect.transform(example)
print('Prognoza: %s\nPrawdopodobieństwo: %.2f%%' %\
      (label[clf.predict(X)[0]], clf.predict_proba(X).max()*100))